In [ ]:
# @title ▶️ Запустить приложение (авто-установка и скрытый UI)
import os, sys, subprocess

def _ensure_deps():
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U',
                           'pandas==2.2.2', 'openpyxl>=3.1,<4', 'gradio>=4.0'])
def _auto_install_pkg():
    # Пытаемся поставить пакет из локальных путей
    candidates = ['.', f'/content/{'tpok003_full'}', '/content/repo']
    for p in candidates:
        if os.path.exists(os.path.join(p, 'pyproject.toml')):
            try:
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-e', p])
                return p
            except Exception as e:
                print('Install failed for', p, '->', e)
    return None

_ensure_deps()
src = _auto_install_pkg()
print('yourpkg installed from:', src)
try:
    from yourpkg.colab_app import launch_app
    launch_app()
except ModuleNotFoundError:
    import sys
    for p in ['.', f'/content/{'tpok003_full'}', '/content/repo']:
        if p not in sys.path:
            sys.path.insert(0, p)
    from yourpkg.colab_app import launch_app
    launch_app()
